In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
import os, sys, pickle, json
import pandas as pd
pd.set_option('display.max_columns', None)

from astropy.coordinates import SkyCoord
from lstchain.reco.utils import  get_effective_time

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ============================ #

# dirs
objects_dir = "/fefs/aswg/workspace/juan.jimenez/systematics/objects"
# dl2 to dl3 information
dl2dl3_dict = "objects/dict_dl2_dl3_MCs.json"
# output file name
output_dict = "objects/dict_dl2.pkl"

# ============================ #

In [2]:
# reading objects
with open(dl2dl3_dict, "r") as json_file:
    dict_runs = json.load(json_file)
    
dict_runs["3319"]

{'dl2': '/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2/process_with_mc_theta_10.0_az_102.199/dl2_LST-1.Run03319.h5',
 'irf': '/fefs/aswg/workspace/juan.jimenez/data/lst1_systematics/_irf/irf_mc_theta_10.0_az_102.199.fits.gz',
 'dl3': '/fefs/aswg/workspace/juan.jimenez/data/lst1_systematics/_dl3/dl3_LST-1.Run03319.fits.gz'}

In [ ]:
%%time
dict_dl2 = {}

for i, run in enumerate(dict_runs.keys()):
    
    print(f"Iterating... {i/len(dict_runs.keys())*100:4.1f}%")
    
    _tab = pd.read_hdf(dict_runs[run]["dl2"], key='dl2/event/telescope/parameters/LST_LSTCam')
    
    # getting only cosmics and applying a small cut in gammaness
    _tab.query("event_type == 32 and gammaness > 0.05", inplace=True)
    
    eff_t, el_t = get_effective_time(_tab)
    
    tmp_dict = {
        "effective_time" : eff_t.to(u.s).value,
        "elapsed_time" : el_t.to(u.s).value,
        "zd" : np.array(90 - np.rad2deg(_tab["alt_tel"])),
        "az" : np.array(np.rad2deg(_tab["az_tel"])),
        "intensity" : np.array(_tab["intensity"])
    }

    dict_dl2[int(run)] = tmp_dict

Iterating...  0.0%
Iterating...  0.9%


In [ ]:
%%time
# Saving the objects
with open(output_dict, 'wb') as f:
    pickle.dump(dict_dl2, f, pickle.HIGHEST_PROTOCOL)
    
# # this to read the objects
# with open(output_dict, 'rb') as f:
#     dict_dl2 = pickle.load(f)